# HW01
# Student: Pravilov Mikhail

In [106]:
import numpy as np

1. (8 points) Implement the kNN algorithm and calculate Leave-one-out error on both datasets for 1 to 10 neighbors.

Firsly, let's load our dataset.

In [107]:
import pandas as pd
dataset_root="../Datasets/"

In [173]:
# spam.csv loading
spam_path = dataset_root + 'spam.csv'
spam_data = pd.read_csv(spam_path)
spam_data = spam_data.values
spam_X = spam_data[:, 0:-1].astype('float64')
spam_y = spam_data[:, -1]
print("spam datapoints shape: " + str(spam_X.shape))
print("spam labels shape: " + str(spam_y.shape))

spam datapoints shape: (4601, 57)
spam labels shape: (4601,)


In [174]:
# cancer.csv loading
cancer_path = dataset_root + 'cancer.csv'
cancer_data = pd.read_csv(cancer_path)
cancer_data = cancer_data.values
cancer_X = cancer_data[:, 1:].astype('float64')
cancer_y = cancer_data[:, 0]
print("cancer datapoints shape: " + str(cancer_X.shape))
print("cancer labels shape: " + str(cancer_y.shape))

cancer datapoints shape: (569, 30)
cancer labels shape: (569,)


Then let's see what sklearn gives us. This will be a baseline to check that our implementation of kNN is correct.

In [148]:
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
k_global = 10
kNNs = [KNeighborsClassifier(n_neighbors=i) for i in range(1, k_global + 1)]

Leave one out error is N-fold cross validation, where N is the number of datapoints

In [152]:
def calculate_loo_error_for_classifiers(classifiers, X, y):
    from sklearn.model_selection import cross_val_score, KFold
    return [1 - cross_val_score(skf, X=X, y=y, cv=KFold(n_splits=y.shape[0]), n_jobs=-1).mean() for skf in classifiers]

In [153]:
# cancer dataset
print("Cancer dataset")
print("<num of neighbours> <loo error>")
cancer_sklearn_loo = calculate_loo_error_for_classifiers(kNNs, cancer_X, cancer_y)
for k, loo_error in enumerate(cancer_sklearn_loo):
    print(str(k + 1) + " " + str(loo_error))

Cancer dataset
<num of neighbours> <loo error>
1 0.08435852372583474
2 0.0773286467486819
3 0.07381370826010547
4 0.07381370826010547
5 0.06678383128295251
6 0.07029876977152905
7 0.06854130052724072
8 0.07029876977152905
9 0.06678383128295251
10 0.07029876977152905


In [154]:
# spam dataset
print("Spam dataset")
print("<num of neighbours> <loo error>")
spam_sklearn_loo = calculate_loo_error_for_classifiers(kNNs, spam_X, spam_y)
for k, loo_error in enumerate(spam_sklearn_loo):
    print(str(k + 1) + " " + str(loo_error))

Spam dataset
<num of neighbours> <loo error>
1 0.16952836339926103
2 0.19061073679634866
3 0.18539447946098675
4 0.19300152140838944
5 0.18582916757226686
6 0.20039121930015213
7 0.19582699413171045
8 0.20104325146707236
9 0.20278200391219303
10 0.2021299717452728


Now let's write our own kNN method.

In [155]:
def get_nearest(k, point_index, X):
    return np.argsort(np.array([np.linalg.norm(X[point_index]-x) for x in X]))[1:k + 1]

def generate_nearest(X, k):
    return np.array([get_nearest(k, i, X) for i in range(X.shape[0])])

def get_class(point_index, nearest_matrix, y, k):
    (nearest_classes, counts) = np.unique(y[nearest_matrix[point_index]][:k], return_counts=True)
    majority_index = np.argmax(counts) 
    return nearest_classes[majority_index]    

In [156]:
def leave_one_out(nearest_matrix, y, k):
    return np.mean([get_class(y_index, nearest_matrix, y, k) != y[y_index] for y_index in range(y.shape[0])])

def leave_one_out_for_all_before_k(X, y, k):
    nearest_matrix = generate_nearest(X, k)
    return [leave_one_out(nearest_matrix, y, i) for i in range(1, k + 1)]

In [157]:
# cancer dataset
print("Cancer dataset")
print("<num of neighbours> <loo error>") 
cancer_loo = leave_one_out_for_all_before_k(cancer_X, cancer_y, k_global)
for k, loo_error in enumerate(cancer_loo):
    print(str(k + 1) + " " + str(loo_error))

Cancer dataset
<num of neighbours> <loo error>
1 0.0843585237258348
2 0.0773286467486819
3 0.07381370826010544
4 0.07381370826010544
5 0.06678383128295255
6 0.070298769771529
7 0.06854130052724078
8 0.070298769771529
9 0.06678383128295255
10 0.070298769771529


In [158]:
# spam dataset
print("Spam dataset")
print("<num of neighbours> <loo error>") 
spam_loo = leave_one_out_for_all_before_k(spam_X, spam_y, k_global)
for k, loo_error in enumerate(spam_loo):
    print(str(k + 1) + " " + str(loo_error))

Spam dataset
<num of neighbours> <loo error>
1 0.16865898717670072
2 0.19039339274070854
3 0.18452510323842644
4 0.19213214518582916
5 0.18539447946098675
6 0.199956531188872
7 0.1958269941317105
8 0.2010432514670724
9 0.20256465985655292
10 0.2019126276896327


In [162]:
print("Cancer dataset. my - sklearn")
print(np.array(cancer_sklearn_loo) - np.array(cancer_loo))

Cancer dataset. my - sklearn
[-5.55111512e-17  0.00000000e+00  2.77555756e-17  2.77555756e-17
 -4.16333634e-17  4.16333634e-17 -5.55111512e-17  4.16333634e-17
 -4.16333634e-17  4.16333634e-17]


In [163]:
print("Spam dataset. my - sklearn")
print(np.array(spam_sklearn_loo) - np.array(spam_loo))

Spam dataset. my - sklearn
[ 8.69376223e-04  2.17344056e-04  8.69376223e-04  8.69376223e-04
  4.34688111e-04  4.34688111e-04 -5.55111512e-17 -2.77555756e-17
  2.17344056e-04  2.17344056e-04]


As you can see, the error is small for the cancer dataset. For spam dataset it is bigger, but I think it is acceptable and does not mean that there is a mistake in my implementation. This is connected probably with the fact that spam dataset is larger and accumulates the error in the distance computations. It is also likely that in the spam dataset classes are quite close to each other, hence more errors in the selection of k neighbors.

In [166]:
cancer_best_k = np.argmin(cancer_loo) + 1
print("Best k for cancer dataset is " + str(cancer_best_k) + " with loo error = " + str(cancer_loo[cancer_best_k - 1]))

Best k for cancer dataset is 5 with loo error = 0.06678383128295255


In [167]:
spam_best_k = np.argmin(spam_loo) + 1
print("Best k for spam dataset is " + str(spam_best_k) + " with loo error = " + str(spam_loo[spam_best_k - 1]))

Best k for spam dataset is 1 with loo error = 0.16865898717670072


If classes are often close to each other then taking big k most likely is bad idea. In the spam dataset this case occurs and best k = 1.

"(2) Scale all features with MinMax scaler to [0,1] and calculate Leave-one-out error on both datasets for 1 to 10 neighbors."

In [177]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def scale_min_max(X, min_val=0, max_val=1):
    return ((X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))) * (max_val - min_val) + min_val

In [182]:
cancer_scaled_X = scale_min_max(cancer_X)
scaler.fit(cancer_X)
cancer_sklearn_scaled_X = scaler.transform(cancer_X)
max_diff = np.max(np.abs(cancer_scaled_X - cancer_sklearn_scaled_X))
print("Max diff in features between my and sklearn implementation " + str(max_diff))

Max diff in features between my and sklearn implementation 2.220446049250313e-16


In [183]:
spam_scaled_X = scale_min_max(spam_X)
scaler.fit(spam_X)
spam_sklearn_scaled_X = scaler.transform(spam_X)
max_diff = np.max(np.abs(spam_scaled_X - spam_sklearn_scaled_X))
print("Max diff in features between my and sklearn implementation " + str(max_diff))

Max diff in features between my and sklearn implementation 1.1102230246251565e-16


Scaler implemented correctly because $10^{-16}$ it is the highest precision of float64.

In [187]:
# cancer dataset
print("Cancer dataset")
print("<num of neighbours> <loo error>") 
cancer_scaled_loo = leave_one_out_for_all_before_k(cancer_scaled_X, cancer_y, k_global)
for k, loo_error in enumerate(cancer_scaled_loo):
    print(str(k + 1) + " " + str(loo_error))

Cancer dataset
<num of neighbours> <loo error>
1 0.04745166959578207
2 0.03866432337434095
3 0.029876977152899824
4 0.028119507908611598
5 0.033391915641476276
6 0.029876977152899824
7 0.029876977152899824
8 0.03163444639718805
9 0.029876977152899824
10 0.029876977152899824


In [189]:
# spam dataset
print("Spam dataset")
print("<num of neighbours> <loo error>") 
spam_scaled_loo = leave_one_out_for_all_before_k(spam_scaled_X, spam_y, k_global)
for k, loo_error in enumerate(spam_scaled_loo):
    print(str(k + 1) + " " + str(loo_error))

Spam dataset
<num of neighbours> <loo error>
1 0.0869376222560313
2 0.10106498587263639
3 0.09497935231471419
4 0.10149967398391654
5 0.09497935231471419
6 0.10258639426211694
7 0.09932623342751576
8 0.10215170615083677
9 0.1019343620951967
10 0.10519452292979788


In [190]:
cancer_scaled_best_k = np.argmin(cancer_scaled_loo) + 1
print("Best k for cancer dataset is " + str(cancer_scaled_best_k) + " with loo error = " + str(cancer_scaled_loo[cancer_best_k - 1]))

Best k for cancer dataset is 4 with loo error = 0.028119507908611598


In [191]:
spam_scaled_best_k = np.argmin(spam_scaled_loo) + 1
print("Best k for spam dataset is " + str(spam_scaled_best_k) + " with loo error = " + str(spam_scaled_loo[spam_best_k - 1]))

Best k for spam dataset is 1 with loo error = 0.0869376222560313


We can see that results after scaling are better approximately in 2 times, not surprisingly.